## ewf-wfp-02-02-02 - Snow Cover Climatology

Snow Cover Climatology

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Snow Cover Climatology'),
                ('abstract', 'Snow Cover Climatology'),
                ('id', 'ewf-wfp-02-02-02')])

### <a name="parameter">Parameter Definition 

In [ ]:
#nameOfRegion = dict([('id', 'nameOfRegion'),
#                     ('value', 'CentralAsia'),
#                     ('title', 'Name of Region'),
#                     ('abstract', 'Name of the region of interest'),
#                     ('minOccurs', '1')])

In [ ]:
indexAggCat = dict([('id', 'indexAggCat'),
                    ('value', 'better-wfp-02-02-01'),
                    ('title', 'indexAggCat'),
                    ('abstract', 'index to access catalog of aggregated land surface temperature time series'),
                    ('minOccurs', '1')])

In [ ]:
apikeyAggCat = dict([('id', 'apikeyAggCat'),
                     ('value', ''),
                     ('title', 'apikeyAggCat'),
                     ('abstract', 'apikey to access indexAggCat catalog'),
                     ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the MDOIS stack of products' identifiers

In [ ]:
# n sdays
input_identifiers = ('26DDF6E2F0A9653EE4878D934A9EF222EA7EB245', '2EC65A47B59254A6E92D996DA4FEFE061896144D')
#input_identifiers = ('2EBB24856B5DBB3F4121FEEA23A6AA9B8782E3BA', 'DB57830AB8372416127C6AD483C5C4334568FB45')

#2EBB24856B5DBB3F4121FEEA23A6AA9B8782E3BA	https://catalog.terradue.com/better-wfp-02-02-01/search?format=atom&uid=2EBB24856B5DBB3F4121FEEA23A6AA9B8782E3BA	https://store.terradue.com/better-wfp-02-02-01/_results/workflows/ec_better_ewf_wfp_02_02_01_ewf_wfp_02_02_01_1_10/run/44b5108a-ea95-11e9-8601-0242ac110026/0138766-181221095105003-oozie-oozi-W/6063aa7b-07c7-4658-8ffc-263ceb55de9c/SCStartSeasonDate_MTerra_h23v05_2015_2016.tif	2015-08-05T00:00:00.0000000Z	2016-07-27T00:00:00.0000000Z	2019-10-09T13:31:44.0879880+00:00	Output SCStartSeasonDate_MTerra_h23v05_2015_2016.tif

#DB57830AB8372416127C6AD483C5C4334568FB45	https://catalog.terradue.com/better-wfp-02-02-01/search?format=atom&uid=DB57830AB8372416127C6AD483C5C4334568FB45	https://store.terradue.com/better-wfp-02-02-01/_results/workflows/ec_better_ewf_wfp_02_02_01_ewf_wfp_02_02_01_1_10/run/f0ae1d30-ee93-11e9-ae3d-0242ac110003/0147245-181221095105003-oozie-oozi-W/1b4cb7b7-0986-4393-882d-fc34d9d11afb/SCStartSeasonDate_MTerra_h23v05_2016_2017.tif	2016-08-04T00:00:00.0000000Z	2017-07-28T00:00:00.0000000Z	2019-10-14T15:50:59.6852860+00:00	Output SCStartSeasonDate_MTerra_h23v05_2016_2017.tif

**Input references**

This is the MODIS stack catalogue references

In [ ]:
input_references = ['https://catalog.terradue.com/better-wfp-02-02-01/search?format=atom&uid={0}'.format(pid) for pid in input_identifiers]

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = "/workspace/dev/ewf-wfp-02-02-02/src/main/app-resources/notebook/libexec"

**Aux folders**

In [ ]:
output_folder = ''

In [ ]:
temp_folder = 'temp'

#### Import Modules

In [ ]:
import os
import shutil

import cioppy
import datetime as dt

import pandas as pd
import geopandas as gpd

import sys
import string
import numpy as np
from scipy import stats
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import copy

import pdb
ciop = cioppy.Cioppy()

#### Auxiliary vars

In [ ]:
check_results = False

#### Auxiliary methods

In [ ]:
def get_input_metadata (input_refs):
    
    # for each product get metadata
    Result_Prod = []
    
    for index,product_ref in enumerate(input_refs):
        
        # since the search is by identifier 
        Result_Prod.append(ciop.search(end_point = product_ref,params =[],output_fields='self,identifier,startdate,enclosure,title,startdate,enddate,wkt',creds='{}:{}'.format(indexAggCat['value'],apikeyAggCat['value']))[0] )
    

    input_metadata = gpd.GeoDataFrame.from_dict(Result_Prod)

    input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
    input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])
    
    return input_metadata

def rm_cfolder(folder):
    #folder = '/path/to/folder'
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
            
    
def get_metadata(filepath):
    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type



def write_output_image(filepath, output_matrix, image_format, data_format, output_projection=None, output_geotransform=None, mask=None, no_data_value=None):
    
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    if mask is not None:
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
        
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)
    
    output.FlushCache()
    


'''
def matrix_sum(mat1, mat2, no_data_value=None):
    if no_data_value is not None:
        if not isinstance(mat1, int):
            mat1[(mat1 == no_data_value)] = 0
        if not isinstance(mat2, int):
            mat2[(mat2 == no_data_value)] = 0
            
            
    msum = mat1 + mat2
        
    msum[(mat1 == 0)] = 0
    msum[(mat2 == 0)] = 0
        
    return msum
    
    
def calc_average(matrix_list, n_years):
    if not isinstance(matrix_list, list):
        return 0
    result = matrix_list[0]
    for i in range(1, n_years):
        result = matrix_sum(result, matrix_list[i])
    
    return np.divide(result, (n_years*1.00))
'''


#
# sums mat1 to mat2
# adds 1 to mat_n_vals where != no_data_value 
#

def matrix_sum(mat1, mat2, mat_n_vals, no_data_value):

    mat2_0and1s = np.zeros(mat2.shape)
    
    mat2_0and1s[mat2 != no_data_value] = 1
    
    
    mat_n_vals = mat2_0and1s;
    
    
    #msum = mat1
    
    msum = copy.deepcopy(mat1)
    
    msum[mat2 != no_data_value] = mat1[mat2 != no_data_value] + mat2[mat2 != no_data_value]
    
    msum[np.logical_and(mat1 == no_data_value, mat2 != no_data_value) ] = mat2[np.logical_and(mat1 == no_data_value, mat2 != no_data_value) ]
    
    msum[np.logical_and(mat1 == no_data_value, mat2 == no_data_value) ] = -9999

    return msum, mat_n_vals


#
# calcs avg of matrix_list
# it takes into account pixels with no_data_values in the time series 
# faster than calc_average_circular_mean
#
def calc_average_faster(matrix_list, no_data_value):
    
    if not isinstance(matrix_list, list):
        return 0
    result = matrix_list[0]
    
  
    mat_n_vals = np.zeros(result.shape)
    mat_n_vals[result != no_data_value] = 1
    
    for i in range(1, len(matrix_list)):
     
        result, mat_n_vals_of_sum = matrix_sum(result, matrix_list[i], mat_n_vals, no_data_value)
        
        mat_n_vals = mat_n_vals + mat_n_vals_of_sum
    

    # to avoid division by 0!!
    mat_n_vals[mat_n_vals == 0] = no_data_value

    result = np.divide(result, mat_n_vals)
    
    # set as no data value pixels that are no data values in all time series
    result[mat_n_vals == no_data_value] = no_data_value
    
    return result


def get_matrix_list(image_list):
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
    return mat_list

#
# calcs avg of matrix_list
# it takes into account pixels with no_data_values in the time series 
# it has option for circular mean
#
def calc_average_circular_mean (snow_matrix_list, no_data_value, apply_circular_mean = False):

    mean_mat = np.ones(snow_matrix_list[0].shape) * no_data_value

    for i in range(snow_matrix_list[0].shape[0]):
        for j in range(snow_matrix_list[0].shape[1]):       
            vals = []
            for k in range(len(snow_matrix_list)):
                v = snow_matrix_list[k][i,j]
                if (v != no_data_value):
                    vals.append(v)
        
            if len(vals) > 0:
                #print(np.mean(vals))
            
                if (apply_circular_mean):
                    mean_mat[i,j] = stats.circmean(vals, high=365, low=1)
                else:
                    mean_mat[i,j] = np.mean(vals)
                    
    return mean_mat


def calc_lta(file_list, apply_circular_mean):
    
    
    if file_list:
        
        n_years = len(file_list)
        agr_period_matrix = get_matrix_list(file_list)
        print('Aggregations converted to matrices')
        
        projection, geotransform, no_data_value, data_type = get_metadata(file_list[0])
        
        #print('no data value' + str(no_data_value))
        
        # change no data value to -9999
        new_no_data_value = -9999
        for i in range(len(agr_period_matrix)):
            agr_period_matrix[i] = agr_period_matrix[i].astype(int)
            agr_period_matrix[i][agr_period_matrix[i] == no_data_value] = new_no_data_value
        no_data_value = new_no_data_value
        
        #print(agr_period_matrix[0])
        
        #print('no data value' + str(no_data_value))
        
        if apply_circular_mean:
            lta = calc_average_circular_mean (agr_period_matrix, no_data_value, apply_circular_mean)
        else:
            lta = calc_average_faster(agr_period_matrix, no_data_value)
        
     
        return lta, projection, geotransform, no_data_value, data_type
    
    else:
        return None, None, None
    
    
    

def write_output(temp_folder, lta, period_start_date, period_end_date, product_type, region, projection, geo_transform, image_format, no_data_value, data_type):
    
 
    output_name = os.path.join(temp_folder, '_'.join(['LTA', product_type, region, str(period_start_date.year), str(period_end_date.year)]) + '.tif' )
    
    write_output_image(output_name, lta, image_format, data_type, projection, geo_transform, no_data_value=no_data_value)
    return output_name

def get_formatted_date(date_str):
    date = datetime.datetime.strftime(date_str, '%Y-%m-%dT00:00:00Z')
    return dat

def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    #first_date = get_formatted_date(first_date)
    #last_date = get_formatted_date(last_date)
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

In [ ]:
#Create folders
#if not os.path.isdir(data_path):
#    os.mkdir(data_path)

if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

##### Get metadata

In [ ]:
input_metadata = get_input_metadata(input_references)
#print(input_metadata)
#file_list = [os.path.join(data_path, in_id.split('/')[-1]) for in_id in input_identifiers]

In [ ]:
apply_circular_mean = False

if 'SeasonDate' in input_metadata['title'].iloc[0]:  
    apply_circular_mean = True

print('Apply circular mean: ' + str(apply_circular_mean))

In [ ]:
file_list = [os.path.join(data_path, os.path.basename(enclosure).split('?')[0]) for enclosure in input_metadata['enclosure']]
file_list

#### Workflow

##### Compute Long Term Averages

In [ ]:
#input_metadata = get_input_metadata(input_references)
#N = nv

#file_list = [os.path.join(data_path, in_id.split('/')[-1]) for in_id in input_identifiers]
#file_list = [os.path.join(data_path, os.path.basename(enclosure).split('?')[0]) for enclosure in input_metadata['enclosure']]

#file_list

lta, projection, geotransform, no_data_value, data_type = calc_lta(file_list, apply_circular_mean)




In [ ]:
lta

In [ ]:
# CHECK
if check_results:
    
    import matplotlib
    import matplotlib.pyplot as plt

    fig = plt.figure()
    plt.imshow(lta)
    plt.show()

#### write output

In [ ]:
import datetime as dt

filename = os.path.splitext(os.path.basename(file_list[0]))[0].split('_')

#startdate = '2015-01-05
#enddate = '2017-01-25'

#startdate = os.path.splitext(os.path.basename(file_list[0]))[0].split('_')[-2]
#enddate = os.path.splitext(os.path.basename(file_list[-1]))[0].split('_')[-1]

#startdate = dt.datetime.strptime(startdate, '%Y')
#enddate = dt.datetime.strptime(enddate, '%Y')

startdate = input_metadata.sort_values('startdate')['startdate'].iloc[0]
enddate = input_metadata.sort_values('enddate')['enddate'].iloc[-1]
region = filename[-3]
prod_type = filename[0] + '_' + filename[-4]
print(filename)
print(region)
print(prod_type)


if lta is not None:
    #pdb.set_trace()
    filename = write_output(output_folder, lta, startdate, enddate, prod_type, region, projection, geotransform, 'GTiff', no_data_value, gdal.GDT_Int32)
    print(filename)
    startdate = input_metadata[input_metadata['startdate'] == min(input_metadata['startdate'])]['startdate'].iloc[0].strftime('%Y-%m-%dT00:00:00Z')
    enddate = input_metadata[input_metadata['enddate'] == max(input_metadata['enddate'])]['enddate'].iloc[0].strftime('%Y-%m-%dT00:00:00Z')
    wkt = input_metadata['wkt'].iloc[0]
    write_properties_file(filename, startdate, enddate, wkt)
    
#lta[310,210]




In [ ]:
region

prod_type

#### Remove temporay files and folders

In [ ]:
rm_cfolder(temp_folder)

os.rmdir(temp_folder)